In [69]:
from utils.utils import make_patches, get_patch_coordinates
from ultralytics import YOLO
from PIL import Image, ImageDraw
import os
import pandas as pd
from copy import deepcopy

# Patches parameters
data_tif_path = "Datasets/Pipeline/Crow_traps_Original"
data_patches_path = "Datasets/Pipeline/Crow_traps_Original/Patches"
patch_size = 224

# Output
data_out_path = "Datasets/Pipeline/Results"
if not os.path.exists(data_out_path):
    os.makedirs(data_out_path)

# Classification parameters
model_cls_path = "runs/classify/train/weights/best.pt"
min_conf_cls = 0.5

# Detection parameters
model_det_path = "runs/detect/train_30_general/weights/best.pt"
min_conf_det = 0.5

# Coordinate parameters
csv_out_path = "Datasets/Pipeline/coordinates.csv"

In [34]:
# Make patches

patches = make_patches(data_tif_path, data_patches_path, patch_size=patch_size)
print("Total patches: ", len(patches))

N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (2, 1, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (2, 1, 1, 224, 224, 3)
N patches  (2, 1, 1, 224, 224, 3)
N patches  (2, 1, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4,

In [64]:
# Classify patches
coordinates_cls = {}
model_cls = YOLO(model_cls_path)  # load model
results_cls = model_cls.predict(source=data_patches_path, imgsz=patch_size, conf=min_conf_cls) 

# Save results
for r in results_cls:
    if r.probs.top1==1:
        # Save imgs
        im = Image.fromarray(r.orig_img[..., ::-1])  # RGB PIL image
        im.save(os.path.join(data_out_path,os.path.basename(r.path)))

        # Update coordinates dictionary
        patch_id, c1, c2 = get_patch_coordinates(r.path, data_tif_path, patch_size=patch_size)
        score = r.probs.top1conf.cpu().numpy()
        coordinates_cls[patch_id] = (f"{c1},{c2}", score, "area")

# Detect crow_traps
coordinates_det = {}
model_det = YOLO(model_det_path)  # load model
results_det = model_det.predict(source=data_patches_path, imgsz=patch_size, conf=min_conf_det) 

for r in results_det:
    for box in r.boxes:
        xyxy = box.xyxy.cpu().numpy()[0]

        # Save imgs
        img = Image.fromarray(r.orig_img[..., ::-1])  # RGB PIL image
        img1 = ImageDraw.Draw(img)   
        img1.rectangle(xyxy, outline ="red") 
        img.save(os.path.join(data_out_path,os.path.basename(r.path)))

        # Update coordinates dictionary
        score = box.conf.cpu().numpy()[0]
        patch_id, c1, c2 = get_patch_coordinates(r.path, data_tif_path, (xyxy[0], xyxy[1]), patch_size=patch_size)
        coordinates_det[patch_id] = (f"{c1},{c2}", score, "crow_trap")


image 1/526 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Crow_traps_Original/Patches/100331_0_0.jpg: 224x224 background 1.00, crow_trap 0.00, 2.8ms
image 2/526 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Crow_traps_Original/Patches/100331_0_1.jpg: 224x224 background 0.99, crow_trap 0.01, 2.6ms
image 3/526 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Crow_traps_Original/Patches/100331_0_2.jpg: 224x224 background 0.94, crow_trap 0.06, 2.7ms
image 4/526 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Crow_traps_Original/Patches/100331_1_0.jpg: 224x224 background 0.98, crow_trap 0.02, 2.2ms
image 5/526 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Crow_traps_Original/Patches/100331_1_1.jpg: 224x224 background 0.96, crow_trap 0.04, 2.2ms
image 6/526 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Crow_traps_Original/Patches/

In [71]:
coordinates = deepcopy(coordinates_cls)

print("Added keys:")
for key in coordinates_det.keys():
    # If not existent, add data to coordinates_cls
    if key not in coordinates_cls.keys():
        coordinates[key] =  coordinates_det[key]
        print(key)
    # If exists, update score
    else:
        c_det, s_det, type_det = coordinates_det[key]
        s_cls = coordinates_cls[key][1]
        coordinates[key] = (c_det, s_cls+s_det, "combined")

Added keys:
100615_2_0
101125_2_1
101442_3_0
101442_3_1
101559_1_2
101679_3_0
101679_3_1
101679_3_2
102237_3_1
102520_1_1
102521_3_2
102523_0_0
102601_3_1
103439_3_2
103519_1_0
103638_3_0
104008_2_1
104268_3_1
104357_0_0
104357_1_2
99911_1_0


In [72]:
df = pd.DataFrame.from_dict(coordinates, orient="index", columns=["Coordinates_QGIS", "Confidence_Score", "Type"])
df = df.sort_values(by=["Confidence_Score"], ascending=False)
df.to_csv(csv_out_path)
df

,Coordinates_QGIS,Confidence_Score,Type
101509_2_1,"-4.000316359247618,57.458171095714384",1.889462,combined
100615_2_1,"-3.03000341807684,55.21009580633509",1.879753,combined
103519_2_1,"-1.7158111716556117,54.18668959818255",1.841625,combined
104272_2_1,"-3.959471825791306,55.421334652761615",1.832651,combined
102237_2_1,"-3.9838182687489634,57.37075913253446",1.825352,combined
101123_2_1,"-3.950443672318618,57.48206085651782",1.80544,combined
101339_2_1,"-3.073161861817624,55.68758219334845",1.784452,combined
103849_2_1,"-2.4318784502106436,55.75007500006888",1.769542,combined
102523_2_1,"-1.9943063686293492,54.20494894327126",1.769349,combined
102520_2_1,"-1.9810942739920872,54.19798497814204",1.763913,combined
